# Name and ID: Hoang Tu Bui - 24005665

# The code executes till the end without any errors

# Original source: 

1. https://medium.com/@liyinxuan0213/step-by-step-double-deep-q-networks-double-dqn-tutorial-from-atari-games-to-bioengineering-dec7e6373896
1. https://stackoverflow.com/questions/26587527/cite-a-paper-using-github-markdown-syntax

# Why Reinforcement Learning is the ML paradigm of choice for this task?

Reinforcement Learning (RL) is well-suited for Atari games like Breakout where an agent must learn to make decisions through trial and error, optimizing its actions based on feedback in the form of rewards. This paradigm is distinct from supervised learning, which relies on pre-labeled data, and unsupervised learning, which seeks to find patterns without any labeled responses ([Shaheen et al., 2025](#deepmind-ateri-2025)).

In Breakout, the agent must learn to control a paddle to bounce a ball and break bricks, receiving rewards only when bricks are broken. This setup presents a challenge where the agent must learn the consequences of its actions over time, making RL an ideal approach . ​

The success of RL in Breakout not only showcases its effectiveness in gaming but also highlights its potential in real-world applications where decision-making under uncertainty and learning from interaction are crucial.​

In summary, RL's framework aligns seamlessly with the challenges presented by Breakout, making it the machine learning paradigm of choice for such tasks.​

# The Environment

1. The game selected: **Breakout**

    The goal is to control a paddle in order to hit a ball toward brick walls at the top of the screen and to break them ([AtariAge - Atari 2600 Manuals (HTML) - Breakout (Atari), 1997-1998](#breakout-description)). The player starts with 5 lives and loses a life if the ball drops. The game terminates and resets when all lives are lost.

1. The input from the environment: A 210x160 RGB image frame from the game screen, where each pixel value is an unsigned 8-bit integer ranging from 0 to 255 ([Implementation/Game environment](#game-environment)).

1. The control settings: four Discrete actions ([Machado et al., 2018](#ale-breakout-doc)):
    - 0:NOOP (No Operation)
    - 1:FIRE
    - 2:RIGHT
    - 3:LEFT

# Implementation

## Game environment

In [ ]:
import ale_py
import gymnasium as gym
import matplotlib.pyplot as plt


gym.register_envs(ale_py)
org_env = gym.make("ALE/Breakout-v5", render_mode="rgb_array")

print(f"Action space: {org_env.action_space}")
print(f"Observation space: {org_env.observation_space}")

org_env.reset()
rgb_arr = org_env.render()

assert rgb_arr is not None, "Rendering empty array"
plt.imshow(rgb_arr)
plt.show()

<a id="obs-preprocess"></a> 
<!-- I noticed that the observation includes redundant information — specifically, the scoreboard at the top, which takes up 20 pixels. Thus they will be cut of using Gymnasium Wrappers.  -->
<!-- Finally, a Normalize wrapper scales pixel value from [0,255] to [0,1]. -->

The preprocess applied to the observations include:

1. RecordEpisodeStatistics: keeps track of cumulative rewards and episode lengths.
1. ResizeObservation((84, 84)): resizes the input frame to 84x84 in order to ensure the input to be in fixed size and reduce computation
1. GrayscaleObservation: reduces input dimensionality from 3 channels (rgb) to 1 (gray) while preserving essential information
1. FrameStackObservation(4): Stacks the last 4 grayscale frames together to provide temporal context
    <!-- 1. EpisodicLifeEnv: Mark the loss of life as end of episode but does not reset the environment. [Shaheen et al., 2025](#deepmind-ateri-2025) found out that it helps value estimation -->
1. MaxAndSkipObservation(skip=4): reduces computation and removes flickering artifacts 
1. FireResetEnv: For certain games like Breakout, after life loss, the game is fixed until firing

In [ ]:
from stable_baselines3.common.atari_wrappers import (
    MaxAndSkipEnv,
    FireResetEnv,
)

# record episode reward and length
env = gym.wrappers.RecordEpisodeStatistics(org_env)
# Resize from 210x160 to 84x84
env = gym.wrappers.ResizeObservation(env, (84, 84))
# Convert rgb to grayscale
env = gym.wrappers.GrayscaleObservation(env)
# Stack 4 frames for temporal information
env = gym.wrappers.FrameStackObservation(env, 4)
# Only change action every 4th frame
env = MaxAndSkipEnv(env, skip=4)
# Reset with FIRE action
env = FireResetEnv(env)

### Observation from the environment

In [ ]:
obs, _ = env.reset()

# plot 4 channels of the observation
fig, axes = plt.subplots(1, 4, figsize=(12, 3))
for i in range(4):
    axes[i].imshow(obs.squeeze()[i], cmap="gray")
    axes[i].axis("off")
plt.tight_layout()
fig.suptitle("Agent's view")
plt.show()

## VANILLA DQN

### Capture and pre-processing of the data

Explained in section [Game environment](#obs-preprocess)

### Network structure

The Deep Q-Network (DQN) implemented here is a convolutional neural network designed to process stacks of image frames, typical in Atari-based reinforcement learning environments. The architecture follows the structure used in the original DeepMind DQN paper by [Mnih et al. (2015)](#deepmind-ateri-2015).
The input to the network is a tensor of shape (4, 84, 84), representing four stacked grayscale frames.

The layers are as follows:

1. **Convolutional Layer 1**

   * `in_channels = 4`, `out_channels = 32`, `kernel_size = 8`, `stride = 4`
   * Output shape: `(32, 20, 20)`

2. **ReLU Activation**

3. **Convolutional Layer 2**

   * `in_channels = 32`, `out_channels = 64`, `kernel_size = 4`, `stride = 2`
   * Output shape: `(64, 9, 9)`

4. **ReLU Activation**

5. **Convolutional Layer 3**

   * `in_channels = 64`, `out_channels = 64`, `kernel_size = 3`, `stride = 1`
   * Output shape: `(64, 7, 7)`

6. **ReLU Activation**

7. **Flatten Layer**

   * Flattens the output from shape `(64, 7, 7)` to a vector of length `3136` (i.e., `64*7*7`)

8. **Fully Connected Layer 1**

   * `in_features = 3136`, `out_features = 512`

9. **ReLU Activation**

10. **Fully Connected Output Layer**

    * `in_features = 512`, `out_features = n_actions`
    * Outputs one Q-value per discrete action.


**In short**, the network extracts spatial features from the input frames using convolutional layers, then maps them through fully connected layers to a Q-value for each possible action.

In [ ]:
from torch import nn


class DQN(nn.Module):
    def __init__(self, n_actions):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(4, 32, 8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3136, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions),
        )

    def forward(self, x):
        return self.network(x / 255.0)  # Normalize to [0, 1]


print(DQN(4))

### Q learning update applied to the weights

In this implementation, Deep Q-Learning is used to train a neural network to approximate the optimal action-value function $Q^*(s, a)$. The network parameters are updated using samples from a Replay Buffer, with the target computed from a fixed target network.

At every train_freq steps (e.g. every 4 steps), if enough transitions are stored, the following update is applied:

1. The weights are updated to minimize the **Huber loss** between the predicted Q-value and the target:

    $$
    \text{Loss} = \text{Huber}(Q(s_t, a_t) - y_t)
    $$

    where the **target** $y_t$ is defined as:

    $$
    y_t = r_t + \gamma \cdot \max_{a'} Q_{\text{target}}(s_{t+1}, a') \cdot (1 - \text{done})
    $$

    Here:

    * $Q(s_t, a_t)$: is the estimated Q-value from the online network for the selected action
    * $Q_{\text{target}}(s_{t+1}, a')$: is the max predicted Q-value from the target network
    * $r_t$: is the clipped reward at time $t$
    * $\gamma$: is the discount factor
    * $\text{done}$: is a binary flag indicating episode termination

1. Mechanics

    1. Sample a minibatch of transitions $(s, a, r, s', \text{done})$ from the replay buffer
    2. Compute current Q-values for $s$ using the online network
    3. Compute target Q-values for $s'$ using the target network (no gradients)
    4. Compute loss between current and target Q-values using `F.huber_loss`
    5. Backpropagate the loss and update the online network weights via Adam optimizer
    6. Every $C$ steps, **synchronize the target network** with the online network


In [ ]:
import copy
from gymnasium import Env
from stable_baselines3.common.buffers import ReplayBuffer
import torch
from torch.optim.adam import Adam
import torch.nn.functional as F
import random
import numpy as np


def calEpsilon(epoch, epsilon_start, epsilon_end, exploration_steps):
    return max(
        (epsilon_end - epsilon_start) / exploration_steps * epoch + epsilon_start,
        epsilon_end,
    )


def DQN_train(
    env: Env,
    model: nn.Module,
    device="cuda",
    buffer_size=10_000,
    n_epoch=30_000,
    train_freq=4,
    batch_size=96,
    gamma=0.99,
    replay_start_size=50,
    epsilon_start=1,
    epsilon_end=0.1,
    exploration_steps=1_000,
    C=10,  # Target network update frequency
    learning_rate=1.25e-4,
    verbose=False,
):

    # Initialize replay memory D to capacity N
    rb = ReplayBuffer(
        buffer_size,
        env.observation_space,
        env.action_space,
        device,
        optimize_memory_usage=True,
        handle_timeout_termination=False,
    )

    # Initialize target action-value function Q_hat
    targetNetwork = copy.deepcopy(model).to(device)
    # Initialize action-value function Q with random weights
    onlineNetwork = model.to(device)
    targetNetwork.load_state_dict(onlineNetwork.state_dict())

    optimizer = Adam(onlineNetwork.parameters(), lr=learning_rate)

    epoch = 0
    total_rewards_list = []
    smoothed_rewards = []
    rewards = []
    total_loss_list = []
    loss_means = []
    losses = []
    best_reward = 0
    bestState = {}

    while epoch <= n_epoch:

        dead = False
        total_rewards = 0

        # Initialise sequence s1 = {x1} and preprocessed sequenced φ1 = φ(s1)
        obs, _ = env.reset()

        for _ in range(random.randint(1, 30)):  # Noop and fire to reset environment
            obs, reward, terminated, truncated, info = env.step(1)

        while not dead:
            current_life = info["lives"]

            epsilon = calEpsilon(epoch, epsilon_start, epsilon_end, exploration_steps)
            if random.random() < epsilon:  # With probability ε select a random action a
                action = np.array(env.action_space.sample())

            else:  # Otherwise select a = max_a Q∗(φ(st), a; θ)
                q_values = onlineNetwork(torch.Tensor(obs).unsqueeze(0).to(device))
                action = np.array(torch.argmax(q_values, dim=1).item())

            # Execute action a in emulator and observe reward rt and image xt+1
            next_obs, reward, terminated, truncated, info = env.step(action)
            dead = terminated or truncated

            # print(f"info: {info}")
            done = np.array(info["lives"] < current_life)

            # Set st+1 = st, at, xt+1 and preprocess φt+1 = φ(st+1)
            real_next_obs = next_obs.copy()

            reward = float(reward)
            total_rewards += reward
            reward = np.sign(reward)  # Reward clipping

            # Store transition (φt, at, rt, φt+1) in D
            rb.add(obs, real_next_obs, action, reward, done, [info])

            obs = next_obs

            if epoch > replay_start_size and epoch % train_freq == 0:
                # Sample random minibatch of transitions (φj , aj , rj , φj +1 ) from D
                data = rb.sample(batch_size)
                with torch.no_grad():
                    max_target_q_value, _ = targetNetwork(data.next_observations).max(
                        dim=1
                    )
                    y = data.rewards.flatten() + gamma * max_target_q_value * (
                        1 - data.dones.flatten()
                    )
                current_q_value = (
                    onlineNetwork(data.observations).gather(1, data.actions).squeeze()
                )

                loss = F.huber_loss(y, current_q_value)

                # Perform a gradient descent
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                losses.append(loss.item())

            # Every C steps reset Q_hat=Q
            if epoch % C == 0:
                targetNetwork.load_state_dict(onlineNetwork.state_dict())

            epoch += 1
            if (epoch % 10_000 == 0) and epoch > 0:
                smoothed_reward = np.mean(rewards) if rewards else 0
                smoothed_rewards.append(smoothed_reward)
                total_rewards_list.append(rewards)
                rewards = []

                loss_mean = np.mean(losses) if losses else 0
                loss_means.append(loss_mean)
                total_loss_list.append(losses)
                losses = []

            if verbose and (epoch % 100_000 == 0):
                print(
                    f"Epoch: {epoch}, Loss: {loss_mean}, Smoothed Reward: {smoothed_reward}"
                )

        rewards.append(total_rewards)

        if total_rewards > best_reward:
            best_reward = total_rewards
            bestState = onlineNetwork.state_dict()

    return bestState, total_rewards_list, total_loss_list

In [ ]:
assert isinstance(env.action_space, gym.spaces.Discrete), "Action space is not discrete"
model = DQN(env.action_space.n)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

trainConfigs = {
    "buffer_size": 100_000,
    "n_epoch": 5_000_000,
    "batch_size": 96,
    "gamma": 0.99,
    "replay_start_size": 5_000,
    "epsilon_start": 1,
    "epsilon_end": 0.1,
    "exploration_steps": 100_000,
    "C": 1_000,
    "learning_rate": 1.25e-4,
}

(
    modelState,
    rewardss,
    losseses,
) = DQN_train(
    env,
    model,
    device=device,
    verbose=True,
    **trainConfigs,
)

### Evaluation

#### Plots

In [ ]:
# unwrap the rewardss
unwrappedRewards = [item for sublist in rewardss for item in sublist]

# plot rewards over episodes
plt.figure(figsize=(12, 5))
plt.plot(unwrappedRewards, label="Episode reward")
# add moving average (1000)
plt.plot(
    np.convolve(unwrappedRewards, np.ones(1000) / 1000, mode="valid"),
    label="Moving average reward",
)
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.title("DQN training rewards")
plt.legend()
plt.show()

In [ ]:
unwrappedLosses = [item for sublist in losseses for item in sublist]
# plot losses
plt.figure(figsize=(12, 5))
plt.plot(unwrappedLosses, label="Loss")
# add moving average (1000)
plt.plot(
    np.convolve(unwrappedLosses, np.ones(1000) / 1000, mode="valid"),
    label="Loss (moving average)",
)
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title("Loss over time")
plt.legend()
plt.show()

#### How does one evaluate the performance of the RL agent

In [ ]:
def play(model: nn.Module, env: Env, device, n_episodes, render):
    model.to(device)
    model.eval()

    frames = []
    for _ in range(n_episodes):
        obs, _ = env.reset()
        dead = False

        # start with fire option
        action = 1

        while not dead:
            obs, _, terminated, truncated, _ = env.step(action)

            dead = terminated or truncated

            if len(frames) < render:
                frames.append(env.render())

            with torch.no_grad():
                obs_tensor = torch.tensor(obs).unsqueeze(0).to(device)

                q_values = model(obs_tensor)
                action = np.array(torch.argmax(q_values, dim=1).item())

            if terminated or truncated:
                action = 1

    return frames

In [ ]:
from matplotlib import animation
from IPython.display import HTML


model.load_state_dict(modelState)

# Play the game
frames = play(model, env, "cpu", n_episodes=3, render=1000)

# Keep 3_rd frames
frames = [frames[i] for i in range(0, len(frames), 3)]

print(f"Drawing {len(frames)} frames")
fig = plt.figure()
plt.axis("off")
im = plt.imshow(frames[0])


def update_frame(frame):
    im.set_data(frame)
    return [im]


ani = animation.FuncAnimation(fig, update_frame, frames=frames, interval=50)
plt.close(fig)
HTML(ani.to_jshtml())

#### Is the agent learning?

## DOUBLE DQN

### Problem

One of the key problems with Vanilla DQN is its tendency to overestimate Q-values during training. This happens because the max operator used in the Q-learning update selects the highest estimated action value, which tends to be overly optimistic due to noise and approximation errors in the value function ([van Hasselt et al., 2016](#double-dqn)).
This overestimation can lead to unstable learning and suboptimal policies, especially in environments with noisy or sparse rewards. As training progresses, these compounded overestimations can mislead the policy into favoring incorrect actions.

### Theoretical solution

Double DQN addresses this overestimation bias by **decoupling action selection from action evaluation**. Instead of using the same network to both select and evaluate the next action, Double DQN uses:

* The **online network** to select the best action (argmax),
* The **target network** to evaluate the Q-value of that action.

Mathematically, the target value becomes:

$$
Q_{\text{target}}(s,a) = r + \gamma \, Q_{\text{target}}\bigl(s', \arg\max_{a'} Q_{\text{online}}(s',a')\bigr)
$$

This separation reduces the chance of overestimating Q-values because the evaluation is less biased by the same noisy estimates used for selection.


In [ ]:
def doubleDQN_train(
    env: Env,
    model: nn.Module,
    device="cuda",
    buffer_size=10_000,
    n_epoch=30_000,
    train_freq=4,
    batch_size=96,
    gamma=0.99,
    replay_start_size=50,
    epsilon_start=1,
    epsilon_end=0.1,
    exploration_steps=1_000,
    C=10,  # Target network update frequency
    learning_rate=1.25e-4,
    verbose=False,
):

    # Initialize replay memory D to capacity N
    rb = ReplayBuffer(
        buffer_size,
        env.observation_space,
        env.action_space,
        device,
        optimize_memory_usage=True,
        handle_timeout_termination=False,
    )

    # Initialize target action-value function Q_hat
    targetNetwork = copy.deepcopy(model).to(device)
    # Initialize action-value function Q with random weights
    onlineNetwork = model.to(device)
    targetNetwork.load_state_dict(onlineNetwork.state_dict())

    optimizer = Adam(onlineNetwork.parameters(), lr=learning_rate)

    epoch = 0
    total_rewards_list = []
    smoothed_rewards = []
    rewards = []
    total_loss_list = []
    loss_means = []
    losses = []
    best_reward = 0
    bestState = {}

    while epoch <= n_epoch:

        dead = False
        total_rewards = 0

        # Initialise sequence s1 = {x1} and preprocessed sequenced φ1 = φ(s1)
        obs, _ = env.reset()

        for _ in range(random.randint(1, 30)):  # Noop and fire to reset environment
            obs, reward, terminated, truncated, info = env.step(1)

        while not dead:
            current_life = info["lives"]

            epsilon = max(
                (epsilon_end - epsilon_start) / exploration_steps * epoch
                + epsilon_start,
                epsilon_end,
            )
            if random.random() < epsilon:  # With probability ε select a random action a
                action = np.array(env.action_space.sample())

            else:  # Otherwise select a = max_a Q∗(φ(st), a; θ)
                q_values = onlineNetwork(torch.Tensor(obs).unsqueeze(0).to(device))
                action = np.array(torch.argmax(q_values, dim=1).item())

            # Execute action a in emulator and observe reward rt and image xt+1
            next_obs, reward, terminated, truncated, info = env.step(action)
            dead = terminated or truncated

            # print(f"info: {info}")
            done = np.array(info["lives"] < current_life)

            # Set st+1 = st, at, xt+1 and preprocess φt+1 = φ(st+1)
            real_next_obs = next_obs.copy()

            reward = float(reward)
            total_rewards += reward
            reward = np.sign(reward)  # Reward clipping

            # Store transition (φt, at, rt, φt+1) in D
            rb.add(obs, real_next_obs, action, reward, done, [info])

            obs = next_obs

            if epoch > replay_start_size and epoch % train_freq == 0:
                # Sample random minibatch of transitions (φj , aj , rj , φj +1 ) from D
                data = rb.sample(batch_size)
                with torch.no_grad():
                    # Use the online network to select the action
                    bestActions = onlineNetwork(data.next_observations).argmax(
                        dim=1, keepdim=True
                    )
                    # Use the target network to evaluate the action
                    nextQ_val = (
                        targetNetwork(data.next_observations)
                        .gather(1, bestActions)
                        .squeeze()
                    )
                y = data.rewards.flatten() + (
                    gamma * nextQ_val * (1 - data.dones.flatten())
                )
                current_q_value = (
                    onlineNetwork(data.observations).gather(1, data.actions).squeeze()
                )

                loss = F.huber_loss(y, current_q_value)

                # Perform a gradient descent
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                losses.append(loss.item())

            # Every C steps reset Q_hat=Q
            if epoch % C == 0:
                targetNetwork.load_state_dict(onlineNetwork.state_dict())

            epoch += 1
            if (epoch % 10_000 == 0) and epoch > 0:
                smoothed_reward = np.mean(rewards) if rewards else 0
                smoothed_rewards.append(smoothed_reward)
                total_rewards_list.append(rewards)
                rewards = []

                loss_mean = np.mean(losses) if losses else 0
                loss_means.append(loss_mean)
                total_loss_list.append(losses)
                losses = []

            if verbose and (epoch % 100_000 == 0):
                print(
                    f"Epoch: {epoch}, Loss: {loss_mean}, Smoothed Reward: {smoothed_reward}"
                )

        rewards.append(total_rewards)

        if total_rewards > best_reward:
            best_reward = total_rewards
            bestState = onlineNetwork.state_dict()

    return bestState, total_rewards_list, total_loss_list

In [ ]:
assert isinstance(env.action_space, gym.spaces.Discrete), "Action space is not discrete"
model = DQN(env.action_space.n)


modelState, rewardss, losseses = doubleDQN_train(
    env,
    model,
    device=device,
    verbose=True,
    **trainConfigs,
)

### Evaluation

In [ ]:
unwrappedRewards = [item for sublist in rewardss for item in sublist]
# plot rewards over episodes
plt.figure(figsize=(12, 5))
plt.plot(unwrappedRewards, label="Episode reward")
# add moving average (1000)
plt.plot(
    np.convolve(unwrappedRewards, np.ones(1000) / 1000, mode="valid"),
    label="Moving average reward",
)
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.title("Double DQN training rewards")
plt.legend()
plt.show()

In [ ]:
unwrappedLosses = [item for sublist in losseses for item in sublist]
# plot losses over steps
plt.figure(figsize=(12, 5))
plt.plot(unwrappedLosses, label="Loss")
# add moving average (1000)
plt.plot(
    np.convolve(unwrappedLosses, np.ones(1000) / 1000, mode="valid"),
    label="Loss (moving average)",
)
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title("Loss over time")
plt.legend()
plt.show()

In [ ]:
from matplotlib import animation
from IPython.display import HTML


model.load_state_dict(modelState)

# Play the game
frames = play(model, env, "cpu", n_episodes=3, render=1000)

# Keep 3_rd frames
frames = [frames[i] for i in range(0, len(frames), 3)]

print(f"Drawing {len(frames)} frames")
fig = plt.figure()
plt.axis("off")
im = plt.imshow(frames[0])


def update_frame(frame):
    im.set_data(frame)
    return [im]


ani = animation.FuncAnimation(fig, update_frame, frames=frames, interval=50)
plt.close(fig)
HTML(ani.to_jshtml())

# Compare DQN and Double DQN

# Added Value

## Hyperparamerter Optimization

[Ken Zheng (2024)](#hyperparam-2024) investigates the impact of hyperparameter tuning on the performance of popular deep reinforcement learning (DRL) algorithms, specifically DDPG, TRPO, and PPO, across three continuous control environments from MuJoCo: HalfCheetah, Hopper, and Walker2d. The motivation stems from the widely acknowledged sensitivity of DRL algorithms to hyperparameter settings, yet many practitioners still rely on default configurations.

To address this, the authors employ BOHB (Bayesian Optimization with Hyperband), an efficient hyperparameter optimization method that balances exploration and exploitation while being budget-aware. They define a search space of nine hyperparameters per algorithm and conduct extensive evaluations to understand how tuning affects final episodic return and sample efficiency. The study compares optimized configurations against default baselines and demonstrates that optimized parameters can significantly boost performance, especially for DDPG and TRPO.

A key finding is that even modest tuning budgets lead to meaningful gains, and blindly using defaults can result in suboptimal performance. However, they also show diminishing returns beyond a certain tuning effort, and that tuning results can be environment-specific. The work emphasizes the importance of systematic tuning in DRL and provides insights into which parameters and search strategies are most impactful.

Below are the comparision of their paramerters and mine:

| **Parameter**            | **Paper's Tuned Value**                             | **My Value**    | **Notes / Comparison**                                                                  |
| ------------------------ | --------------------------------------------------- | ----------------- | --------------------------------------------------------------------------------------- |
| `batch_size`             | `84`                                                | `96`              | Similar. Marginal difference — likely low impact unless memory is a concern.            |
| `gamma`                  | `0.95298`                                           | `0.99`            | I discount future rewards more — they found a slightly more short-term focus better.  |
| `learning_rate`          | `6.6566e-5`                                         | `1.25e-4`         | Mine makes training faster; theirs is more conservative and possibly more stable.          |
| `replay_start_size`      | `50000`                                             | `5000`              |  they delay learning until buffer is filled more. Prevents early bias. |
| `target_update`          | `2500`                                              | `1000` (my `C`)   | Their target network updates less frequently. This might add more stability.           |
| `exploration_steps`      | - | `100_000`            | Their exploration comprises of 2 linear phrases 1->0.1->0.01, over 1 million then 22 million of frames while mine is one linear decaying over 100_000 frames.   |


In [ ]:
tunnedConfigs = {
    "buffer_size": 100_000,
    "n_epoch": 5_000_000,
    "batch_size": 84,
    "gamma": 0.95298,
    "replay_start_size": 50_000,
    "epsilon_start": 0.147,
    "epsilon_end": 0.1,
    "exploration_steps": 100_000,
    "C": 2500,
    "learning_rate": 6.6566e-5,
}


# their way of calculating decay epsilon
# full epsilon range is from 1.0 → 0.1 → 0.01, transitioning across:
# Step 2 to 1,000,000: Decay from 1.0 → 0.1
# Step 1,000,000 to 22,000,000: Decay from 0.1 → 0.01
# After 22M steps: Fixed ε = 0.01


def calEpsilon(epoch, epsilon_start, epsilon_end, exploration_steps):
    current_step = epoch
    startpoint = 2
    end = 0.1
    kneepoint = 1000000
    final_eps = 0.01
    final_knee_point = 22000000

    if current_step < startpoint:
        return 1
    mid_seg = end + np.maximum(
        0, (1 - end) - (1 - end) / kneepoint * (current_step - startpoint)
    )
    if not final_eps:
        return mid_seg
    else:
        if final_eps and final_knee_point and (current_step < kneepoint):
            return mid_seg
        else:
            return final_eps + (end - final_eps) / (final_knee_point - kneepoint) * (
                final_knee_point - current_step
            )


assert isinstance(env.action_space, gym.spaces.Discrete), "Action space is not discrete"
model = DQN(env.action_space.n)

modelState, rewardss, losseses = doubleDQN_train(
    env,
    model,
    device=device,
    verbose=True,
    **tunnedConfigs,
)

In [ ]:
unwrappedRewards = [item for sublist in rewardss for item in sublist]
# plot rewards over episodes
plt.figure(figsize=(12, 5))
plt.plot(unwrappedRewards, label="Episode reward")
# add moving average (1000)
plt.plot(
    np.convolve(unwrappedRewards, np.ones(1000) / 1000, mode="valid"),
    label="Moving average reward",
)
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.title("Double DQN training rewards")
plt.legend()
plt.show()

In [ ]:
unwrappedLosses = [item for sublist in losseses for item in sublist]
# plot losses over steps
plt.figure(figsize=(12, 5))
plt.plot(unwrappedLosses, label="Loss")
# add moving average (1000)
plt.plot(
    np.convolve(unwrappedLosses, np.ones(1000) / 1000, mode="valid"),
    label="Loss (moving average)",
)
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title("Loss over time")
plt.legend()
plt.show()

## Prioritised Experience Replay

In [ ]:
from torchrl.data import PrioritizedReplayBuffer, ListStorage


def prioritisedDoubleDQN_train(
    env: Env,
    model: nn.Module,
    device="cuda",
    buffer_size=10_000,
    n_epoch=30_000,
    train_freq=4,
    batch_size=96,
    gamma=0.99,
    replay_start_size=50,
    epsilon_start=1,
    epsilon_end=0.1,
    exploration_steps=1_000,
    C=10,  # Target network update frequency
    learning_rate=1.25e-4,
    verbose=False,
):

    # Initialize replay memory D to capacity N
    rb = PrioritizedReplayBuffer(
        storage=ListStorage(buffer_size),
        alpha=0.7,
        beta=0.5,
        batch_size=batch_size,
        pin_memory=True,
    )

    # Initialize target action-value function Q_hat
    targetNetwork = copy.deepcopy(model).to(device)
    # Initialize action-value function Q with random weights
    onlineNetwork = model.to(device)
    targetNetwork.load_state_dict(onlineNetwork.state_dict())

    optimizer = Adam(onlineNetwork.parameters(), lr=learning_rate)

    epoch = 0
    total_rewards_list = []
    smoothed_rewards = []
    rewards = []
    total_loss_list = []
    loss_means = []
    losses = []
    best_reward = 0
    bestState = {}

    while epoch <= n_epoch:

        dead = False
        total_rewards = 0

        # Initialise sequence s1 = {x1} and preprocessed sequenced φ1 = φ(s1)
        obs, _ = env.reset()

        for _ in range(random.randint(1, 30)):  # Noop and fire to reset environment
            obs, reward, terminated, truncated, info = env.step(1)

        while not dead:
            current_life = info["lives"]

            epsilon = max(
                (epsilon_end - epsilon_start) / exploration_steps * epoch
                + epsilon_start,
                epsilon_end,
            )
            if random.random() < epsilon:  # With probability ε select a random action a
                action = np.array(env.action_space.sample())

            else:  # Otherwise select a = max_a Q∗(φ(st), a; θ)
                q_values = onlineNetwork(torch.Tensor(obs).unsqueeze(0).to(device))
                action = np.array(torch.argmax(q_values, dim=1).item())

            # Execute action a in emulator and observe reward rt and image xt+1
            next_obs, reward, terminated, truncated, info = env.step(action)
            dead = terminated or truncated

            # print(f"info: {info}")
            done = np.array(info["lives"] < current_life)

            # Set st+1 = st, at, xt+1 and preprocess φt+1 = φ(st+1)
            real_next_obs = next_obs.copy()

            reward = float(reward)
            total_rewards += reward
            reward = np.sign(reward)  # Reward clipping

            # Store transition (φt, at, rt, φt+1) in D
            rbData = {
                "obs": torch.tensor(obs),
                "next_obs": torch.tensor(real_next_obs),
                "action": torch.tensor(action),
                "reward": torch.tensor(reward),
                "done": torch.tensor(done),
                # "info": torch.tensor(info),
            }
            rb.add(rbData)

            obs = next_obs

            if epoch > replay_start_size and epoch % train_freq == 0:
                # Sample random minibatch of transitions (φj , aj , rj , φj +1 ) from D
                rbData, rbInfo = rb.sample(batch_size, True)
                for k, v in rbData.items():
                    rbData[k] = v.float().to(device)

                with torch.no_grad():
                    # Use the online network to select the action
                    bestActions = onlineNetwork(rbData["next_obs"]).argmax(
                        dim=1, keepdim=True
                    )
                    # Use the target network to evaluate the action
                    nextQ_val = (
                        targetNetwork(rbData["next_obs"])
                        .gather(1, bestActions)
                        .squeeze()
                    )
                y = rbData["reward"].flatten() + (
                    gamma * nextQ_val * (1 - rbData["done"].flatten())
                )
                current_q_value = (
                    onlineNetwork(rbData["obs"])
                    .gather(1, rbData["action"].to(torch.int64).unsqueeze(-1))
                    .squeeze()
                )

                loss = F.huber_loss(y, current_q_value)

                # cal the priority
                tdErr = (y - current_q_value).abs()
                rb.update_priority(rbInfo["index"], tdErr)

                # Perform a gradient descent
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                losses.append(loss.item())

            # Every C steps reset Q_hat=Q
            if epoch % C == 0:
                targetNetwork.load_state_dict(onlineNetwork.state_dict())

            epoch += 1
            if (epoch % 10_000 == 0) and epoch > 0:
                smoothed_reward = np.mean(rewards) if rewards else 0
                smoothed_rewards.append(smoothed_reward)
                total_rewards_list.append(rewards)
                rewards = []

                loss_mean = np.mean(losses) if losses else 0
                loss_means.append(loss_mean)
                total_loss_list.append(losses)
                losses = []

            if verbose and (epoch % 100_000 == 0):
                print(
                    f"Epoch: {epoch}, Loss: {loss_mean}, Smoothed Reward: {smoothed_reward}"
                )

        rewards.append(total_rewards)

        if total_rewards > best_reward:
            best_reward = total_rewards
            bestState = onlineNetwork.state_dict()

    return bestState, total_rewards_list, total_loss_list

In [ ]:
assert isinstance(env.action_space, gym.spaces.Discrete), "Action space is not discrete"
model = DQN(env.action_space.n)

modelState, rewardss, losseses = prioritisedDoubleDQN_train(
    env,
    model,
    device=device,
    verbose=True,
    **trainConfigs,
)

In [ ]:
unwrappedRewards = [item for sublist in rewardss for item in sublist]
# plot rewards over episodes
plt.figure(figsize=(12, 5))
plt.plot(unwrappedRewards, label="Episode reward")
# add moving average (1000)
plt.plot(
    np.convolve(unwrappedRewards, np.ones(1000) / 1000, mode="valid"),
    label="Moving average reward",
)
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.title("Double DQN training rewards")
plt.legend()
plt.show()

In [ ]:
unwrappedLosses = [item for sublist in losseses for item in sublist]
# plot losses over steps
plt.figure(figsize=(12, 5))
plt.plot(unwrappedLosses, label="Loss")
# add moving average (1000)
plt.plot(
    np.convolve(unwrappedLosses, np.ones(1000) / 1000, mode="valid"),
    label="Loss (moving average)",
)
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title("Loss over time")
plt.legend()
plt.show()

## Discussions

[Mnih et al. (2015)](#deepmind-ateri-2015) discussed performance across different Atari games and explained why Vanilla DQN works well or poorly in certain games. 
For games like Breakout, Pong, Robotank, ... where the agent reaches **Superhuman** performance because the environment sends frequent and immediate rewards.
In contrast, games like Seaquest and Pitfall feature sparse rewards, requiring long sequences of correct actions before any feedback is received — leading the DQN agent to perform worse than a human, and sometimes even worse than a random policy.

One fundamental weakness of Vanilla DQN is its tendency to overestimate action values due to the use of the max operator in Q-learning updates. 
This overestimation bias significantly impacts games like Seaquest, where noisy reward signals can mislead the value function. 
To address this, [van Hasselt et al. (2016)](#double-dqn) introduced Double DQN, which decouples action selection and evaluation by using separate networks. This significantly reduces overestimation bias and leads to more stable and accurate learning, especially in environments where DQN struggles.

# References

1. <a id="breakout-description"></a>
AtariAge—Atari 2600 Manuals (HTML)—Breakout (Atari). (1997-1998). Retrieved May 5, 2025, from https://atariage.com/manual_html_page.php?SoftwareID=889

1. <a id="ale-breakout-doc"></a>
Machado, M. C., Bellemare, M. G., Talvitie, E., Veness, J., Hausknecht, M. J., & Bowling, M. (2018). Revisiting the Arcade Learning Environment: Evaluation Protocols and Open Problems for General Agents. Journal of Artificial Intelligence Research, 61, 523–562.
https://ale.farama.org/environments/breakout/

1. <a id="deepmind-ateri-2025"></a>
Shaheen, A., Badr, A., Abohendy, A., Alsaadawy, H., & Alsayad, N. (2025). Reinforcement Learning in Strategy-Based and Atari Games: A Review of Google DeepMinds Innovations (No. arXiv:2502.10303). arXiv. https://doi.org/10.48550/arXiv.2502.10303

1. <a id="deepmind-ateri-2015"></a>
Mnih, V., Kavukcuoglu, K., Silver, D., Rusu, A. A., Veness, J., Bellemare, M. G., Graves, A., Riedmiller, M., Fidjeland, A. K., Ostrovski, G., Petersen, S., Beattie, C., Sadik, A., Antonoglou, I., King, H., Kumaran, D., Wierstra, D., Legg, S., & Hassabis, D. (2015). Human-level control through deep reinforcement learning. Nature, 518(7540), 529–533. https://doi.org/10.1038/nature14236

1. <a id="hyperparam-2024"></a>
Zheng, Ken. (2024). Hyperparameter Optimization for Deep Reinforcement Learning: An Atari Breakout Case Study. 10.58445/rars.1635. 

1. <a id="double-dqn"></a>
van Hasselt, H., Guez, A., & Silver, D. (2016). Deep Reinforcement Learning with Double Q-Learning. Proceedings of the AAAI Conference on Artificial Intelligence, 30(1). https://doi.org/10.1609/aaai.v30i1.10295
